# COMP 432 - BIXI

In [1]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm
import time
# Removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Load BIXI data

In [2]:
zf = ZipFile('./data/bixi/biximontrealrentals2019-33ea73.zip')
bixi_2019_04 = pd.read_csv(zf.open('OD_2019-04.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_05 = pd.read_csv(zf.open('OD_2019-05.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_06 = pd.read_csv(zf.open('OD_2019-06.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_07 = pd.read_csv(zf.open('OD_2019-07.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_08 = pd.read_csv(zf.open('OD_2019-08.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_09 = pd.read_csv(zf.open('OD_2019-09.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_10 = pd.read_csv(zf.open('OD_2019-10.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_stations = pd.read_csv(zf.open('Stations_2019.csv'), dtype={'name': 'str'})

/Users/simon/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
bixi_2019_04.head()

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1
1,2019-04-14 07:59:31,6411,2019-04-14 08:09:18,6411,587,1
2,2019-04-14 07:59:55,6097,2019-04-14 08:12:11,6036,736,1
3,2019-04-14 07:59:57,6310,2019-04-14 08:27:58,6345,1680,1
4,2019-04-14 08:00:37,7029,2019-04-14 08:14:12,6250,814,0


In [4]:
bixi_2019_stations.head()

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478,-73.570
1,4000,Jeanne-d'Arc / Ontario,45.550,-73.542
2,4001,Graham / Brookfield,45.520,-73.630
3,4002,Graham / Wicksteed,45.517,-73.640
4,5002,St-Charles / Montarville,45.534,-73.515


## Load weather data

In [5]:
# Note: Use Date/Time column instead of Time column
weather_2019_04 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_04-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_05 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_05-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_06 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_06-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_07 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_07-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_08 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_08-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_09 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_09-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_10 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_10-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})

In [6]:
weather_2019_04.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 00:00:00,2019,4,1,00:00,-1.500,...,nan,24.100,nan,100.750,nan,nan,nan,-7.000,nan,Snow
1,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 01:00:00,2019,4,1,01:00,-1.700,...,nan,24.100,nan,100.790,nan,nan,nan,-9.000,nan,Cloudy
2,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 02:00:00,2019,4,1,02:00,-2.900,...,nan,24.100,nan,100.860,nan,nan,nan,-9.000,nan,<NA>
3,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 03:00:00,2019,4,1,03:00,-3.900,...,nan,24.100,nan,100.920,nan,nan,nan,-10.000,nan,<NA>
4,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 04:00:00,2019,4,1,04:00,-4.800,...,nan,24.100,nan,101.030,nan,nan,nan,-11.000,nan,Clear


## Merging all data together

### BIXI data

Merging all 2019 bixi trips together

In [7]:
bixi_2019_all = pd.concat([bixi_2019_04, bixi_2019_05, bixi_2019_06, bixi_2019_07, bixi_2019_08, bixi_2019_09, bixi_2019_10])
bixi_2019_all

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1
1,2019-04-14 07:59:31,6411,2019-04-14 08:09:18,6411,587,1
2,2019-04-14 07:59:55,6097,2019-04-14 08:12:11,6036,736,1
3,2019-04-14 07:59:57,6310,2019-04-14 08:27:58,6345,1680,1
4,2019-04-14 08:00:37,7029,2019-04-14 08:14:12,6250,814,0
...,...,...,...,...,...,...
617190,2019-10-31 23:52:45,6901,2019-11-01 00:05:15,6063,749,1
617191,2019-10-31 23:53:59,6218,2019-11-01 00:23:19,6269,1760,1
617192,2019-10-31 23:55:26,7149,2019-11-01 00:11:33,6338,966,1
617193,2019-10-31 23:56:28,6199,2019-11-01 00:09:13,6003,764,1


Append station data to all Bixi Trips

***May take a while to run**

In [8]:
bixi_2019_all_with_stations = bixi_2019_all.merge(bixi_2019_stations, left_index=True, left_on='start_station_code', right_on='Code').drop('Code', 1).rename({'name': 'start_name', 'latitude': 'start_latitude', 'longitude': 'start_longitude'}, axis=1).merge(bixi_2019_stations, left_index=True, left_on='end_station_code', right_on='Code').drop('Code', 1).rename({'name': 'end_name', 'latitude': 'end_latitude', 'longitude': 'end_longitude'}, axis=1)
bixi_2019_all_with_stations.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude
45,2019-10-24 08:46:31,6044,2019-10-24 09:02:15,6040,943,0,Roy / St-André,45.522,-73.570,St-Jacques / St-Pierre,45.503,-73.559
361,2019-08-27 18:33:58,6206,2019-08-27 18:36:16,6411,137,1,Prince-Arthur / du Parc,45.511,-73.575,Clark / Prince-Arthur,45.513,-73.573
450,2019-05-28 17:45:42,6247,2019-05-28 17:50:33,6912,291,1,St-Dominique / St-Zotique,45.532,-73.611,Métro Beaubien (de Chateaubriand / Beaubien),45.535,-73.604
501,2019-07-12 15:39:49,6193,2019-07-12 16:08:51,7036,1742,0,de l'Esplanade / Fairmount,45.521,-73.597,Labadie / du Parc,45.520,-73.597
29,2019-06-09 01:59:25,6729,2019-06-09 02:07:09,6021,463,1,St-André / Ste-Catherine,45.516,-73.558,de l'Hôtel-de-Ville / Sherbrooke,45.515,-73.568


### Weather

Merging weather data

In [9]:
weather_2019_all = pd.concat([weather_2019_04, weather_2019_05, weather_2019_06, weather_2019_07, weather_2019_08, weather_2019_09, weather_2019_10])
weather_2019_all.sample(10)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
88,-73.740,45.470,MONTREAL INTL A,7025251,2019-06-04 16:00:00,2019,6,4,16:00,15.500,...,nan,24.100,nan,100.770,nan,nan,nan,nan,nan,Cloudy
315,-73.740,45.470,MONTREAL INTL A,7025251,2019-09-14 03:00:00,2019,9,14,03:00,16.400,...,nan,24.100,nan,101.470,nan,nan,nan,nan,nan,Rain
350,-73.740,45.470,MONTREAL INTL A,7025251,2019-05-15 14:00:00,2019,5,15,14:00,13.500,...,nan,48.300,nan,100.400,nan,nan,nan,nan,nan,<NA>
13,-73.740,45.470,MONTREAL INTL A,7025251,2019-08-01 13:00:00,2019,8,1,13:00,26.600,...,nan,48.300,nan,101.580,nan,27.000,nan,nan,nan,Mostly Cloudy
197,-73.740,45.470,MONTREAL INTL A,7025251,2019-09-09 05:00:00,2019,9,9,05:00,9.800,...,nan,24.100,nan,102.100,nan,nan,nan,nan,nan,<NA>
658,-73.740,45.470,MONTREAL INTL A,7025251,2019-05-28 10:00:00,2019,5,28,10:00,11.800,...,nan,24.100,nan,100.870,nan,nan,nan,nan,nan,Cloudy
676,-73.740,45.470,MONTREAL INTL A,7025251,2019-08-29 04:00:00,2019,8,29,04:00,18.400,...,nan,24.100,nan,100.470,nan,nan,nan,nan,nan,Mostly Cloudy
150,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-07 06:00:00,2019,4,7,06:00,2.400,...,nan,24.100,nan,101.940,nan,nan,nan,nan,nan,<NA>
461,-73.740,45.470,MONTREAL INTL A,7025251,2019-06-20 05:00:00,2019,6,20,05:00,17.300,...,nan,24.100,nan,100.070,nan,nan,nan,nan,nan,<NA>
118,-73.740,45.470,MONTREAL INTL A,7025251,2019-10-05 22:00:00,2019,10,5,22:00,7.800,...,nan,24.100,nan,102.140,nan,nan,nan,nan,nan,Mostly Cloudy


### BIXI + Weather

Merging weather data (Tempeture and weather) with BIXI data

In [10]:
bixi_2019_all_with_stations_and_weather = bixi_2019_all_with_stations.merge(weather_2019_all[['Date/Time', 'Weather', 'Temp (°C)']], left_on=bixi_2019_all_with_stations['start_date'].dt.floor('H'), right_on='Date/Time').drop('Date/Time', 1).rename({'Weather': 'start_weather', 'Temp (°C)': 'start_temp'}, axis=1)
bixi_2019_all_with_stations_and_weather.sample(10)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude,start_weather,start_temp
1464671,2019-09-26 17:30:56,7056,2019-09-26 17:41:01,7144,604,1,Bibliothèque de Verdun (Brown / Bannantyne),45.448,-73.578,Hickson / Wellington,45.465,-73.567,<NA>,17.100
3288070,2019-06-21 19:22:27,6387,2019-06-21 19:25:54,6394,206,1,Métro Joliette (Joliette / Hochelaga),45.547,-73.552,Valois / Ontario,45.547,-73.545,Cloudy,23.600
3345769,2019-05-24 08:02:37,7036,2019-05-24 08:15:47,6035,790,0,Labadie / du Parc,45.520,-73.597,Viger / Chenneville,45.505,-73.561,Rain,13.000
3558800,2019-06-09 20:24:59,6374,2019-06-09 20:42:27,6396,1047,0,15e avenue / Masson,45.554,-73.571,Métro Pie-IX (Pierre-de-Coubertin / Pie-IX),45.554,-73.552,<NA>,26.000
2940362,2019-05-01 08:40:55,6097,2019-05-01 08:43:55,6092,179,1,Ste-Catherine / St-Marc,45.493,-73.580,Crescent / Ste-Catherine,45.497,-73.576,<NA>,4.400
134387,2019-09-11 20:55:29,6108,2019-09-11 21:03:42,6002,493,1,Logan / Fullum,45.528,-73.551,Ste-Catherine / Dezery,45.539,-73.541,<NA>,16.000
4082223,2019-06-14 13:07:29,7037,2019-06-14 13:10:51,6003,201,1,Prince-Arthur / Ste-Famille,45.512,-73.575,Clark / Evans,45.511,-73.568,Cloudy,15.700
1076627,2019-08-05 17:11:55,6073,2019-08-05 17:21:13,6222,558,1,de Maisonneuve / Aylmer (ouest),45.505,-73.571,de l'Esplanade / du Mont-Royal,45.518,-73.588,<NA>,27.300
2535556,2019-08-02 12:48:59,6269,2019-08-02 13:00:07,6902,667,1,Wurtele / de Rouen,45.538,-73.553,Montcalm / Ontario,45.521,-73.562,<NA>,27.500
3313712,2019-06-30 20:20:26,6417,2019-06-30 20:23:27,6396,181,1,de la Salle / Hochelaga,45.554,-73.547,Métro Pie-IX (Pierre-de-Coubertin / Pie-IX),45.554,-73.552,<NA>,20.400


## Data Cleaning

### Checking if there are missing data
Only the weather data is missing some values

In [11]:
bixi_2019_all_with_stations_and_weather.isnull().sum()

start_date                  0
start_station_code          0
end_date                    0
end_station_code            0
duration_sec                0
is_member                   0
start_name                  0
start_latitude              0
start_longitude             0
end_name                    0
end_latitude                0
end_longitude               0
start_weather         3459073
start_temp               1494
dtype: int64

### Checking the contents of the "Weather" column
Some of the weather values contains more than 1 condition. We'll have to take this into account while training.

In [12]:
bixi_2019_all_with_stations_and_weather['start_weather'].value_counts()

Mostly Cloudy                              633845
Mainly Clear                               400336
Cloudy                                     304650
Clear                                      281318
Rain                                       165817
Rain Showers                               102038
Rain,Fog                                    20620
Drizzle,Fog                                 17388
Fog                                         16463
Drizzle                                     16140
Thunderstorms,Rain Showers                  11358
Moderate Rain,Fog                            4062
Moderate Rain Showers                        3378
Rain,Rain Showers                            1437
Moderate Rain                                1202
Thunderstorms,Rain                            793
Thunderstorms,Moderate Rain Showers,Fog       635
Thunderstorms,Moderate Rain Showers           612
Thunderstorms,Heavy Rain Showers              577
Thunderstorms                                 274


We will have to find a way to split the weather conditions into smaller parts. 

The best way to do so is to create new features to the data that represent the weather and set the flag to either 0 or 1

Here are the unique weather conditions:
- Mostly Cloudy
- Mainly Clear
- Cloudy
- Clear
- Rain
- Rain Showers
- Fog
- Drizzle
- Moderate Rain Showers
- Moderate Rain
- Heavy Rain Showers
- Thunderstorms

Also, more than half of the data does not have a weather appended to it. I'm not sure if this will affect the results or not. 

## Expanding data

### Weather flags
First, we create new features for the weather data and set the appropriate flag for all trip data

***May take a while to run**

In [13]:
# Using "temp" variable so I can display the results and debug before overwriting the original variable
temp = bixi_2019_all_with_stations_and_weather

temp = temp.join(temp['start_weather'].str.get_dummies(','))

bixi_2019_all_with_stations_and_weather = temp

bixi_2019_all_with_stations_and_weather.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Drizzle,Fog,Heavy Rain Showers,Mainly Clear,Moderate Rain,Moderate Rain Showers,Mostly Cloudy,Rain,Rain Showers,Thunderstorms
3738910,2019-06-19 13:16:17,6749,2019-06-19 13:25:27,6073,549,1,St-Jacques / St-Laurent,45.506,-73.556,de Maisonneuve / Aylmer (ouest),...,0,0,0,0,0,0,0,0,0,0
1836550,2019-05-24 17:28:27,6906,2019-05-24 17:34:09,6260,341,1,Métro Rosemont (de St-Vallier / Rosemont),45.532,-73.598,Cartier / Dandurand,...,0,0,0,0,0,0,0,0,0,0
1393056,2019-06-14 19:56:01,6316,2019-06-14 20:05:46,6420,585,1,Swail / Decelles,45.497,-73.619,Métro Snowdon (de Westbury / Queen-Mary),...,0,0,0,0,0,0,1,0,0,0
1169348,2019-04-29 14:24:58,6100,2019-04-29 14:44:09,6064,1150,1,Mackay / de Maisonneuve,45.497,-73.579,Métro Peel (de Maisonneuve / Stanley),...,0,0,0,0,0,0,0,0,0,0
2651546,2019-07-27 13:19:52,6170,2019-07-27 13:29:39,6227,587,1,de Mentana / Laurier,45.530,-73.588,de l'Esplanade / Laurier,...,0,0,0,0,0,0,0,0,0,0


### Splitting start_date to year, month, day, hour, minute, second columns
Also adding seconds since midnight for simpler normalization

In [14]:
temp = bixi_2019_all_with_stations_and_weather

temp['start_year'] = temp['start_date'].dt.year
temp['start_month'] = temp['start_date'].dt.month
temp['start_day'] = temp['start_date'].dt.day
temp['start_hour'] = temp['start_date'].dt.hour
temp['start_minute'] = temp['start_date'].dt.minute
temp['start_second'] = temp['start_date'].dt.second
temp['start_seconds_since_midnight'] = (temp['start_date'] - pd.to_datetime(temp['start_date'].dt.strftime('%Y-%m-%d') + ' ' + '00:00:00')).astype('timedelta64[s]')

bixi_2019_all_with_stations_and_weather = temp

bixi_2019_all_with_stations_and_weather.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Rain,Rain Showers,Thunderstorms,start_year,start_month,start_day,start_hour,start_minute,start_second,start_seconds_since_midnight
199042,2019-07-02 13:41:02,6039,2019-07-02 13:44:41,6052,219,1,McGill / des Récollets,45.501,-73.559,de la Commune / King,...,0,0,0,2019,7,2,13,41,2,49262.000
5393314,2019-06-02 10:18:24,6902,2019-06-02 10:21:51,6729,207,1,Montcalm / Ontario,45.521,-73.562,St-André / Ste-Catherine,...,0,1,0,2019,6,2,10,18,24,37104.000
2328220,2019-06-06 09:34:22,6254,2019-06-06 09:44:51,7025,629,1,Boyer / Bélanger,45.540,-73.609,St-Dominique / Bernard,...,0,0,0,2019,6,6,9,34,22,34462.000
1989248,2019-07-10 08:17:09,6209,2019-07-10 08:25:35,6059,505,1,Milton / Clark,45.513,-73.571,de la Cathédrale / René-Lévesque,...,0,0,0,2019,7,10,8,17,9,29829.000
4136576,2019-05-19 18:38:52,6060,2019-05-19 18:47:00,6014,488,1,Belmont / Robert-Bourassa,45.501,-73.566,St-Denis / René-Lévesque,...,0,1,0,2019,5,19,18,38,52,67132.000


### Adding number of trips in a +/- 30 minutes time frame of that particular station

This will serve as our "y" parameter, aka the value we are trying to predict

In [123]:
# _m = bixi_2019_all_with_stations_and_weather['start_month'] == 8
# _d = bixi_2019_all_with_stations_and_weather['start_day'] == 14
# _h = bixi_2019_all_with_stations_and_weather['start_hour'] == 8
# _c = bixi_2019_all_with_stations_and_weather['start_station_code'] == 6176
# _c2 = bixi_2019_all_with_stations_and_weather['start_station_code'] == 6014

# temp = bixi_2019_all_with_stations_and_weather[_m & _d & (_c | _c2)].sample(10)

#2019-09-22 15:29:16 	6152
#2019-04-14 08:49:42 	6176
        
temp = bixi_2019_all_with_stations_and_weather
len(temp)

5442288

Slow. 500+ hours to run

In [16]:
# # I couldn't figure out how to vectorize this...

# with tqdm(total=len(temp)) as pbar: # Progress bar
#     for i, row in temp.iterrows():
#         # Get the current row's station code
#         target_station_code = row['start_station_code']
#         target_start_date = row['start_date']

#         # Retrieve all the data for that particular station code
#         df_station = temp[(temp['start_station_code'] == target_station_code)]
        
#         # Count how many trips were started +/- 30min of the start time
#         sum_count = df_station['start_date'].between(
#             target_start_date + pd.Timedelta('-30 minutes'),
#             target_start_date + pd.Timedelta('30 minutes')
#         ).sum()

#         #Assign sum to new trip_count column
#         temp.loc[i,'trip_count'] = sum_count
        
#         #Update progress bar
#         pbar.update(1)

# temp.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')

In [17]:
# processed = pd.read_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
# processed.sort_values(by='start_date')

A little bit better but still slow. Maybe 250 hours?

In [18]:
# def count_trips(row):
#     station = temp['start_station_code'] == row['start_station_code']
#     t_date = row['start_date']
#     t_start = row['start_date'] + pd.Timedelta('-30 minutes')
#     t_end = row['start_date'] + pd.Timedelta('30 minutes')
#     return len(temp[station & (t_start < t_date) & (t_date < t_end)])

In [19]:
# start_time = time.time()
# temp['trip_count'] = temp.apply(count_trips, axis=1)
# print("--- %s seconds ---" % (time.time() - start_time))
# temp.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')

# temp.sort_values(by='start_date')

In [20]:
# processed = pd.read_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
# processed.sort_values(by='start_date')

Testing

In [ ]:
group = temp.groupby('start_station_code')

with tqdm(total=len(group)) as pbar: # Progress bar
    trip_count_temp = []
    for code, df in group:
        trip_count_group = df.apply(lambda x: 
                                    df['start_date'].between(
                                        x['start_date'] + pd.Timedelta('-30 minutes'),
                                        x['start_date'] + pd.Timedelta('30 minutes')).sum(), axis=1)
        trip_count_temp.append(trip_count_group)
        pbar.update(1)

print("Converting... This may take a while but it should be faster than the previous one")
trip_count = pd.DataFrame(pd.DataFrame(trip_count_temp).T.apply(lambda x: x.sum(), axis=1), columns=['trip_count'])
print("Done")

trip_count

print("Concat to temp...")
tc = pd.concat([temp, trip_count], axis=1, sort=False)
print("Done")

print("Saving to disk...")
tc.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
print("Done")

In [ ]:
processed = pd.read_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
processed.sort_values(by='start_date')